In [113]:
import pandas as pd
from datetime import datetime


In [114]:
data = pd.read_csv('../old_data/velo.csv',sep=";")
data.head()


,Numéro de boucle,Jour,00,01,02,03,04,05,06,07,...,20,21,22,23,Total,Probabilité de présence d'anomalies,Jour de la semaine,Boucle de comptage,Date formatée,Vacances
0,665,2024-12-31,2.0,2.0,2.0,0.0,1.0,1.0,7.0,18.0,...,27.0,13.0,7.0,1.0,414,NaN,2,0665 - Bonduelle vers Nord,2024-12-31,Vacances de Noël
1,673,2024-12-31,5.0,5.0,1.0,2.0,2.0,3.0,3.0,31.0,...,14.0,6.0,8.0,4.0,396,NaN,2,0673 - Pont de Pornic vers Sud,2024-12-31,Vacances de Noël
2,880,2024-12-31,13.0,8.0,7.0,5.0,3.0,2.0,9.0,28.0,...,27.0,25.0,10.0,4.0,826,NaN,2,0880 - Madeleine vers Nord,2024-12-31,Vacances de Noël
3,944,2024-12-31,0.0,0.0,1.0,0.0,0.0,0.0,1.0,5.0,...,0.0,1.0,0.0,1.0,108,NaN,2,0944 - Magellan vers Est,2024-12-31,Vacances de Noël
4,946,2024-12-31,1.0,1.0,2.0,0.0,0.0,1.0,4.0,8.0,...,14.0,1.0,2.0,4.0,297,NaN,2,0946 - Magellan vers Est,2024-12-31,Vacances de Noël


In [115]:
colonnes_a_conserver = [
    'Numéro de boucle',
    'Jour',
    'Total',
    'Jour de la semaine',
    'Boucle de comptage',
    'Vacances'
]
df = data[colonnes_a_conserver]

In [116]:
df = df.rename(columns={
    'Numéro de boucle': 'code_boucle',
    'Boucle de comptage': 'nom_station',
     'Jour': 'date',
     'Jour de la semaine': 'jour_semaine',
     'Vacances': 'vacances',
     'Total': 'total',
}

)
df['date']=pd.to_datetime(df['date'])
df.head()

,code_boucle,date,total,jour_semaine,nom_station,vacances
0,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël
1,673,2024-12-31,396,2,0673 - Pont de Pornic vers Sud,Vacances de Noël
2,880,2024-12-31,826,2,0880 - Madeleine vers Nord,Vacances de Noël
3,944,2024-12-31,108,2,0944 - Magellan vers Est,Vacances de Noël
4,946,2024-12-31,297,2,0946 - Magellan vers Est,Vacances de Noël


In [117]:
df.isna()

,code_boucle,date,total,jour_semaine,nom_station,vacances
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
24883,False,False,False,False,False,False
24884,False,False,False,False,False,False
24885,False,False,False,False,False,False
24886,False,False,False,False,False,False


In [118]:
df_station = df[['code_boucle','nom_station']]
df_station = df_station.drop_duplicates()

df_station['id_station'] = df_station.reset_index().index + 1


In [119]:
df_station.to_csv("../data/station.csv", index=False) 

In [120]:
data1=pd.read_csv("../old_data/meteo.csv",sep=",")
data1.head()

,time,temperature_2m (°C),precipitation (mm),wind_speed_10m (km/h),cloud_cover (%),wind_gusts_10m (km/h),apparent_temperature (°C)
0,2024-01-01T00:00,5.0,0.0,14.1,100,23.4,1.3
1,2024-01-01T01:00,5.4,0.0,13.7,100,24.1,1.8
2,2024-01-01T02:00,5.1,0.0,14.0,99,23.4,1.4
3,2024-01-01T03:00,4.7,0.0,14.5,68,24.5,0.8
4,2024-01-01T04:00,4.4,0.0,14.0,14,24.1,0.6


In [121]:
df['date']=pd.to_datetime(df['date'])
df_date = df[['date','vacances']]
df_date = df_date.drop_duplicates()
df_date = df_date.reset_index(drop=True)
df_date['id_date'] = df_date.index +1
df_date.head()


,date,vacances,id_date
0,2024-12-31,Vacances de Noël,1
1,2024-12-30,Vacances de Noël,2
2,2024-12-29,Vacances de Noël,3
3,2024-12-28,Vacances de Noël,4
4,2024-12-27,Vacances de Noël,5


In [122]:
df_date['mois'] = df_date['date'].dt.month
df_date['jour'] = df_date['date'].dt.day
df_date['name_day'] = df_date['date'].dt.dayofweek + 1

df_date.head()



,date,vacances,id_date,mois,jour,name_day
0,2024-12-31,Vacances de Noël,1,12,31,2
1,2024-12-30,Vacances de Noël,2,12,30,1
2,2024-12-29,Vacances de Noël,3,12,29,7
3,2024-12-28,Vacances de Noël,4,12,28,6
4,2024-12-27,Vacances de Noël,5,12,27,5


In [123]:
df_date.to_csv("../data/temps.csv", index=False) 

In [124]:
data1=pd.read_csv("../old_data/meteo.csv")
data1.head()

,time,temperature_2m (°C),precipitation (mm),wind_speed_10m (km/h),cloud_cover (%),wind_gusts_10m (km/h),apparent_temperature (°C)
0,2024-01-01T00:00,5.0,0.0,14.1,100,23.4,1.3
1,2024-01-01T01:00,5.4,0.0,13.7,100,24.1,1.8
2,2024-01-01T02:00,5.1,0.0,14.0,99,23.4,1.4
3,2024-01-01T03:00,4.7,0.0,14.5,68,24.5,0.8
4,2024-01-01T04:00,4.4,0.0,14.0,14,24.1,0.6


In [125]:
data1=data1.rename(columns={
    'temperature_2m (°C)':'temperature',
    'precipitation (mm)':'precipitation',
    'wind_speed_10m (km/h)':'wind_speed',
    'cloud_cover (%)':'cloud_cover',
    'wind_gusts_10m (km/h)':'wind_gusts',
    'apparent_temperature (°C)':'apparent_temperature',
    'time':'date'



})
data1.head()

,date,temperature,precipitation,wind_speed,cloud_cover,wind_gusts,apparent_temperature
0,2024-01-01T00:00,5.0,0.0,14.1,100,23.4,1.3
1,2024-01-01T01:00,5.4,0.0,13.7,100,24.1,1.8
2,2024-01-01T02:00,5.1,0.0,14.0,99,23.4,1.4
3,2024-01-01T03:00,4.7,0.0,14.5,68,24.5,0.8
4,2024-01-01T04:00,4.4,0.0,14.0,14,24.1,0.6


In [126]:
data1['date'] = pd.to_datetime(data1['date'])
data1['date'] = data1['date'].dt.date
df_meteo_jour = data1.groupby('date').max(numeric_only=True).reset_index()
df_meteo_jour['id_meteo'] = df_meteo_jour.index + 1
df_meteo_jour = df_meteo_jour[['id_meteo', 'date', 'temperature', 'precipitation', 'wind_speed', 'cloud_cover', 'wind_gusts', 'apparent_temperature']]

df_meteo_jour.to_csv("../data/meteo.csv", index=False)

df_meteo_jour.head()



,id_meteo,date,temperature,precipitation,wind_speed,cloud_cover,wind_gusts,apparent_temperature
0,1,2024-01-01,7.4,0.6,19.7,100,36.0,4.0
1,2,2024-01-02,8.0,1.9,20.2,100,35.6,4.2
2,3,2024-01-03,10.6,2.9,27.8,100,47.9,6.3
3,4,2024-01-04,7.3,0.4,33.1,100,61.9,3.5
4,5,2024-01-05,1.0,0.9,21.3,100,36.7,-2.9


In [127]:
data1['date']=pd.to_datetime(data1['date'])

In [128]:
df.head()

,code_boucle,date,total,jour_semaine,nom_station,vacances
0,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël
1,673,2024-12-31,396,2,0673 - Pont de Pornic vers Sud,Vacances de Noël
2,880,2024-12-31,826,2,0880 - Madeleine vers Nord,Vacances de Noël
3,944,2024-12-31,108,2,0944 - Magellan vers Est,Vacances de Noël
4,946,2024-12-31,297,2,0946 - Magellan vers Est,Vacances de Noël


In [129]:
df_faits_final = pd.merge(df, data1, on='date', how='left')
df_faits_final.head()


,code_boucle,date,total,jour_semaine,nom_station,vacances,temperature,precipitation,wind_speed,cloud_cover,wind_gusts,apparent_temperature
0,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël,1.3,0.0,15.9,83,33.5,-3.1
1,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël,1.6,0.0,14.8,40,31.0,-2.7
2,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël,1.5,0.0,14.9,12,30.6,-2.8
3,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël,1.2,0.0,14.7,41,30.2,-3.1
4,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël,1.1,0.0,14.4,100,29.9,-3.1


In [130]:
df_faits = df_faits_final.merge(df_station, on=['code_boucle', 'nom_station'], how='left')
# Convertir la colonne 'date' au bon format partout
df_faits['date'] = pd.to_datetime(df_faits['date'])
df_meteo_jour['date'] = pd.to_datetime(df_meteo_jour['date'])
df_date['date'] = pd.to_datetime(df_date['date'])

df_faits = df_faits.merge(df_meteo_jour[['date', 'id_meteo']], on='date', how='left')
df_faits = df_faits.merge(df_date[['date', 'id_date']], on='date', how='left')

df_faits.head()




,code_boucle,date,total,jour_semaine,nom_station,vacances,temperature,precipitation,wind_speed,cloud_cover,wind_gusts,apparent_temperature,id_station,id_meteo,id_date
0,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël,1.3,0.0,15.9,83,33.5,-3.1,1,366,1
1,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël,1.6,0.0,14.8,40,31.0,-2.7,1,366,1
2,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël,1.5,0.0,14.9,12,30.6,-2.8,1,366,1
3,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël,1.2,0.0,14.7,41,30.2,-3.1,1,366,1
4,665,2024-12-31,414,2,0665 - Bonduelle vers Nord,Vacances de Noël,1.1,0.0,14.4,100,29.9,-3.1,1,366,1


In [131]:
df_faits_clean = df_faits[['id_station', 'id_date', 'id_meteo', 'total']]
df_faits_clean.head()



,id_station,id_date,id_meteo,total
0,1,1,366,414
1,1,1,366,414
2,1,1,366,414
3,1,1,366,414
4,1,1,366,414


In [132]:
df_faits_clean = df_faits_clean.reset_index(drop=True)
df_faits_clean['id_fait'] = df_faits_clean.index + 1

# Réorganiser les colonnes
df_faits_clean = df_faits_clean[['id_fait', 'id_station', 'id_date', 'id_meteo', 'total']]


In [133]:
df_faits_clean.to_csv("../data/fait_comptage.csv", index=False)
